In [ ]:
import numpy as np
from numpy.random import random

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
WINDOW_HEIGHT = 100
WINDOW_WIDTH = 100
BLOCK_SIZE = 10
STATES = {'free': 0, 'occupied': 1, 'unknown': 2, 'fov': 3, 'agent': -1}
COLORS = {0: WHITE, 1: RED, 2: BLACK, 3: BLUE, -1: GREEN}

class Map:
    def __init__(self, width=WINDOW_WIDTH, height=WINDOW_HEIGHT, block_size=BLOCK_SIZE, states=STATES, colors=COLORS):
        self.width = width
        self.height = height
        self.block_size = block_size
        self.grid_vect = np.zeros((height // block_size, width // block_size), dtype=int)
        self.states = states
        self.colors = colors

    def draw(self):
        for x in range(0, self.height, self.block_size):
            for y in range(0, self.width, self.block_size):
                state = self.grid_vect[x // self.block_size][y // self.block_size]
                # print(str(state) + '', end="")
            # print('')

class MapHandler:
    def __init__(self, width=WINDOW_WIDTH, height=WINDOW_HEIGHT, block_size=BLOCK_SIZE, states=STATES, colors=COLORS):
        self.width = width
        self.height = height
        self.block_size = block_size
        self.grid_map = Map(width, height, block_size, states, colors)
        self.agent_map = Map(width, height, block_size, states, colors)
        self.init_maps()

    def init_maps(self, occ_threshold=0.1, known_threshold=0.1, start_pos=((WINDOW_HEIGHT // BLOCK_SIZE) // 2, (WINDOW_WIDTH // BLOCK_SIZE) // 2)):
        for x in range(0, self.height, self.block_size):
            for y in range(0, self.width, self.block_size):

                prob_occ = random() < occ_threshold
                prob_unk = random() < known_threshold
                if prob_occ:    
                    self.grid_map.grid_vect[x // self.block_size][y // self.block_size] = self.grid_map.states['occupied']
                if prob_unk:
                    self.agent_map.grid_vect[x // self.block_size][y // self.block_size] = self.grid_map.grid_vect[x // self.block_size][y // self.block_size]
                else:
                    self.agent_map.grid_vect[x // self.block_size][y // self.block_size] = self.agent_map.states['unknown']
        # Add Agent
        self.grid_map.grid_vect[start_pos[0]][start_pos[1]] = self.grid_map.states['agent']
        self.agent_map.grid_vect[start_pos[0]][start_pos[1]] = self.agent_map.states['agent']
    
    def update_map(self, fov=3):
        agent_idx = np.where(self.agent_map.grid_vect == self.agent_map.states['agent'])
        agent_idx = (agent_idx[1][0], agent_idx[0][0])
        dir_h = {}
        dir_v = {}
        if agent_idx[1] > fov:
            dir_h['neg'] = -1
        if agent_idx[1] < len(self.agent_map.grid_vect) - fov - 1:
            dir_h['pos'] = 1

        if agent_idx[0] > fov:
            dir_v['neg'] = -1
        if agent_idx[0] < len(self.agent_map.grid_vect[0]) - fov - 1:
            dir_v['pos'] = 1

        print(f"Starting FOV Map with agent index: {agent_idx}")
        dir_h, cost_h, gain_h = self.fov_map(dir_h, agent_idx, fov=fov, axis=0)
        dir_v, cost_v, gain_v = self.fov_map(dir_v, agent_idx, fov=fov, axis=1)
            

    def fov_map(self, directions, agent_idx, fov, axis):
        cost = {'pos': 0, 'neg': 0}
        gain = {'pos': 0, 'neg': 0}
        for d in directions:
            # print(f'Start loop for direction: {d}')
            if d == 'pos':
                mov_range = range(agent_idx[axis] + 1, agent_idx[axis] + fov)
            else:
                mov_range = range(agent_idx[axis] - 1, agent_idx[axis] - fov, -1)
            stop_iter = False

            p = 1
            for i in mov_range:
                # print(f'Start fov position: {i}')
                a_map = self.agent_map.grid_vect[:, agent_idx[axis]] if axis == 1 else self.agent_map.grid_vect[agent_idx[axis], :]
                g_map = self.grid_map.grid_vect[:, agent_idx[axis]] if axis == 1 else self.grid_map.grid_vect[agent_idx[axis], :]
                if a_map[i] == 2:
                    gain[d] += 1
                a_map[i] = 3
                # print("Start Drawing")
                self.agent_map.draw()
                if g_map[i] == 1:
                    directions[d] = p-1 if d == 'pos' else -p+1
                    stop_iter = True
                else:
                    directions[d] = p if d == 'pos' else -p

                cost[d] += 1
                print(f'End loop for direction: {d}, cost: {cost[d]}, gain: {gain[d]}')
                if stop_iter:
                    print(f'Found obstacle in direction {d}, stopping iteration')
                    break
                p += 1
            start_point = agent_idx[axis] + 1 if d == 'pos' else agent_idx[axis] + directions[d] - 1
            # print(start_point, directions[d])
            end_point = start_point + directions[d] + 1 if d == 'pos' else agent_idx[axis]
            # print(start_point, end_point)
            a_elements = self.agent_map.grid_vect[:, agent_idx[axis]] if axis == 1 else self.agent_map.grid_vect[agent_idx[axis], :]
            g_elements = self.grid_map.grid_vect[:, agent_idx[axis]] if axis == 1 else self.grid_map.grid_vect[agent_idx[axis], :]
            a_elements[start_point:end_point] = g_elements[start_point:end_point]
        self.agent_map.draw()
        return directions, cost, gain

In [236]:
map_handler = MapHandler()
print('Real Map:')
map_handler.grid_map.draw()

Real Map:
0100000111
0000010000
0000000000
0010000000
0001000000
10100-10000
1000000100
1100000000
0000000000
0000000010


In [237]:
map_handler.agent_map.draw()

0222222222
2222020222
0222222202
2222022222
2002222222
22202-12022
2222022202
1222222222
2222222222
2222022022


In [238]:
map_handler.update_map(fov=3)

Starting FOV Map with agent index: (5, 5)
Start loop for direction: neg
Start fov position: 4
Start Drawing
0222222222
2222020222
0222222202
2222022222
2002222222
22203-12022
2222022202
1222222222
2222222222
2222022022
End loop for direction: neg, cost: 1, gain: 1
Start fov position: 3
Start Drawing
0222222222
2222020222
0222222202
2222022222
2002222222
22233-12022
2222022202
1222222222
2222222222
2222022022
End loop for direction: neg, cost: 2, gain: 1
2 -2
2 5
Start loop for direction: pos
Start fov position: 6
Start Drawing
0222222222
2222020222
0222222202
2222022222
2002222222
22100-13022
2222022202
1222222222
2222222222
2222022022
End loop for direction: pos, cost: 1, gain: 1
Start fov position: 7
Start Drawing
0222222222
2222020222
0222222202
2222022222
2002222222
22100-13322
2222022202
1222222222
2222222222
2222022022
End loop for direction: pos, cost: 2, gain: 1
6 2
6 9
0222222222
2222020222
0222222202
2222022222
2002222222
22100-10002
2222022202
1222222222
2222222222
222202202

In [239]:
map_handler.agent_map.draw()

0222222222
2222020222
0222202202
2222002222
2002202222
22100-10002
2222002202
1222202222
2222202222
2222022022


In [6]:
import numpy as np
prova = {'left': 3, 'right': 2, 'bb': 5, 'down': 0}
(np.max(list(prova.values())), list(prova.keys())[np.argmax(list(prova.values()))])

(5, 'bb')

In [ ]:
import numpy as np
t = np.array([[1,2,3], [4,5,6], [7,8,9]])
t.size

AttributeError: 'list' object has no attribute 'size'

In [ ]:
def dijkstra_path(vertices, neighbors, start_node, end_node):
    unvisited = {v: float('inf') for v in vertices}
    unvisited[start_node] = 0
    print(unvisited)
    previous = {v: None for v in vertices}
    print(previous)
    print(end_node)

    while unvisited:
        current = min(unvisited, key=unvisited.get)
        if current == end_node:
            break
        for neighbor in neighbors.get(current, []):
            if neighbor in unvisited:
                alt = unvisited[current] + 1  # Assuming all edges have weight 1
                if alt < unvisited[neighbor]:
                    unvisited[neighbor] = alt
                    previous[neighbor] = current
        unvisited.pop(current)

    path = []
    while previous[end_node] is not None:
        path.append(end_node)
        end_node = previous[end_node]
    path.append(start_node)
    return path[::-1]

def matrix_graph(map, start_pos, end_pos):
    binary_matrix = np.array([[1 if map[i][j] != STATES['occupied'] and map[i][j] != STATES['unknown'] else 0 for j in range(len(map[0]))] for i in range(len(map))])
    binary_matrix[end_pos[0]][end_pos[1]] = 1

    vertices = set()
    neighbors = {}
    for i in range(len(binary_matrix)):
        for j in range(len(binary_matrix[0])):
            if binary_matrix[i][j] == 1:
                label = i * len(binary_matrix[0]) + j
                vertices.add(label)
                neighbors[label] = set()
                if i > 0 and binary_matrix[i-1][j] == 1:  # Up
                    neighbors[label].add((i-1) * len(binary_matrix[0]) + j)
                if i < len(binary_matrix) - 1 and binary_matrix[i+1][j] == 1:  # Down
                    neighbors[label].add((i+1) * len(binary_matrix[0]) + j)
                if j > 0 and binary_matrix[i][j-1] == 1:  # Left
                    neighbors[label].add(i * len(binary_matrix[0]) + (j-1))
                if j < len(binary_matrix[0]) - 1 and binary_matrix[i][j+1] == 1:  # Right
                    neighbors[label].add(i * len(binary_matrix[0]) + (j+1))
    start_node = start_pos[0] * len(binary_matrix[0]) + start_pos[1]
    end_node = end_pos[0] * len(binary_matrix[0]) + end_pos[1]
    return vertices, neighbors, start_node, end_node

def find_frontier(mp):
    idx_list = []
    # Find frontier
    for i in range(len(mp)):
        for j in range(len(mp[0])):
            if mp[i][j] != STATES['unknown']:
                continue
            test_frontier = False
            if i > 0 and mp[i-1][j] == STATES['free']:  # Up
                test_frontier = True

            if i < len(mp) - 1 and mp[i+1][j] == STATES['free']:  # Down
                test_frontier = True

            if j > 0 and mp[i][j-1] == STATES['free']:  # Left
                test_frontier = True

            if j < len(mp[0]) - 1 and mp[i][j+1] == STATES['free']:  # Right
                test_frontier = True
            if test_frontier:
                idx_list.append((i, j))

    return idx_list


STATES = {'free': 0, 'occupied': 1, 'unknown': 2, 'fov': 3, 'agent': -1}
t = np.array([[0,0,0,0,2], [2,0,0,0,1], [2,0,0,1,2], [2, 0, 0, 2, 2]])
start_pos=(1,1)
end_pos=(3,3)
v, n, start_node, end_node = matrix_graph(t, start_pos, end_pos)
print(t)
print(v)
print(n)
print(start_node)
print(end_node)

[[0 0 0 0 2]
 [2 0 0 0 1]
 [2 0 0 1 2]
 [2 0 0 2 2]]
{0, 1, 2, 3, 6, 7, 8, 11, 12, 16, 17, 18}
{0: {1}, 1: {0, 2, 6}, 2: {1, 3, 7}, 3: {8, 2}, 6: {1, 11, 7}, 7: {8, 2, 12, 6}, 8: {3, 7}, 11: {16, 12, 6}, 12: {17, 11, 7}, 16: {17, 11}, 17: {16, 18, 12}, 18: {17}}
6
18


In [24]:
d = dijkstra_path(v, n, start_node, end_node)
print(d)

{0: inf, 1: inf, 2: inf, 3: inf, 6: 0, 7: inf, 8: inf, 11: inf, 12: inf, 16: inf, 17: inf, 18: inf}
{0: None, 1: None, 2: None, 3: None, 6: None, 7: None, 8: None, 11: None, 12: None, 16: None, 17: None, 18: None}
18
[6, 7, 12, 17, 18]


In [25]:
idx_list, unknown_dir = find_frontier(t)
print(idx_list)
print(unknown_dir)

[(0, 4), (1, 0), (2, 0), (3, 0), (3, 3)]
{'right': 1, 'down': 1, 'left': -1}
